In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit, least_squares
import matplotlib.pyplot as plt
from scipy.special import kn, iv
from scipy.integrate import quad
from scipy.integrate import tplquad

In [12]:
# Read the CSV file and extract data from the first column, skipping the first row
data = pd.read_csv("D:/ACADEMICS/Coding/Comp Phy/term_paper/pion_data.csv", usecols=[0,3], skiprows=[0])
lim_length = 40
# Convert the extracted data to a list
data_list = data.values.tolist()
data_list = np.array(data_list)
pT = np.array(data_list)[:,0].flatten()
nine = np.array(data_list)[:,1].flatten()

# changing dtype of pT to float
pT = pT[:lim_length].astype(float)
nine = nine[:lim_length].astype(float)

print(pT.shape)
print(nine.shape)
#print(pT)

(40,)
(40,)


In [13]:
#defining the bgbw
def boltzmann_gibbs_blast_wave(p_T, m, T, beta_s, R, n, norm):
    #defining the function integration
    def integrand(r, p_T, m_t, T, beta_s, n):
        '''
        input parameters
        - r : radial distance
        - p_T : transverse momentum array
        - m_t : transverse mass
        - T : temperature
        - beta_s : surface flow velocity
        - n : profile shape
        '''
        # transverse radial flow velocity
        beta = beta_s * (r / R) ** n
        #finding flow profile
        rho = np.arctanh(beta)
        #argument for modified bessel's function of 0th order
        arg1 = p_T * np.sinh(rho) / T
        #argument for modified bessel's function of 1st order
        arg2 = m_t * np.cosh(rho) / T
        return r * kn(1, arg2) * iv(0, arg1)
    #integrating to find transverse momenta
    dN_dpT = np.zeros_like(p_T)
    for i,p in enumerate(p_T):
        m_t = np.sqrt(m**2 + p**2)
        result, _ = quad(integrand, 0, R, args=(p, m_t, T, beta_s, n))
        dN_dpT[i] = result * m_t * norm
    return dN_dpT


In [14]:
#defining tbw
def tsallis_blast_wave(p_T, Y,  m, T, q, R, beta_s, n, norm):

    def integrand(r, phi, y, p_T, m_T, T, q, beta_s, n):
        # rho = np.arctanh(beta_s * ((r / R) ** n)) # radial beta
        rho = np.arctanh(beta_s * (1 + 1/(n + 1))) # average beta

        weight_factor = 1 + (((m_T * np.cosh(y) * np.cosh(rho)) - (p_T * np.sinh(rho) * np.cos(phi)))*((q - 1) / T))
        weight_factor = weight_factor ** (-1 / (q - 1))
        weight_factor = weight_factor * r * np.cosh(y)
        return weight_factor
    
    dN_dpT = np.zeros_like(p_T)
    for i,p in enumerate(p_T):
        m_T = np.sqrt(m**2 + p**2)
        #print(m_T)
        result, _ = tplquad(integrand, -Y, Y, -np.pi, np.pi, 0, R, args=(p, m_T, T, q, beta_s, n))
        # print(f'{result:.100f}')
        dN_dpT[i] = result * m_T  * norm

    return dN_dpT

In [15]:
m_0_pi = 0.13957
T_0 = 1.2676e-1
beta_s_0 = 4.387e-1
beta = 4.387e-1
n_0_bg = 0.5
R_0 = 0.01 #update
q_0 = 1.060
Y_0 = 0.5
n_0_ts = 1
norm_bg = 1e6
norm_ts = 1e6


#dN_dpT_bg = boltzmann_gibbs_blast_wave(pT, m_0, T_0, beta_s_0, R_0, n_0_bg, norm_bg)
#dN_dpT_ts = tsallis_blast_wave(pT, Y_0,  m_0, T_0, q_0, R_0, beta, n_0_ts, norm_ts)

In [16]:
### ---------- fitting the data to the models ---------- ###

# parameter bounds for boltzmann gibbs blast wave model
mass_bounds_bg = (m_0_pi, m_0_pi+1e-5)
temp_bounds_bg = (0, 1)
beta_bounds_bg = (0, 1)
n_0_bg_bounds_bg = (0, 2)
R_0_bounds_bg = (1e-5, 1)
norm_bg_bounds_bg = (0, 1e10)

all_bounds_bg = ([mass_bounds_bg[0], temp_bounds_bg[0], beta_bounds_bg[0], R_0_bounds_bg[0], n_0_bg_bounds_bg[0], norm_bg_bounds_bg[0]],[mass_bounds_bg[1], temp_bounds_bg[1], beta_bounds_bg[1], R_0_bounds_bg[1], n_0_bg_bounds_bg[1], norm_bg_bounds_bg[1]])

#parameter bounds for tsallis blast wave model
Y_0_bounds_ts = (0.5,0.5+1e-5)
mass_bounds_ts = (m_0_pi,m_0_pi+1e-5)
q_0_bounds_ts = (1.001,2)
temp_bounds_ts = (0.1,1)
R_0_bounds_ts = (1e-5, 1)
beta_bounds_ts = (0.00001, 1)
n_0_ts_bounds_ts = (1, 1+1e-5)
norm_ts_bounds_ts = (0, 1e10)

all_bounds_ts =( [Y_0_bounds_ts[0], mass_bounds_ts[0], temp_bounds_ts[0], q_0_bounds_ts[0], R_0_bounds_ts[0], beta_bounds_ts[0], n_0_ts_bounds_ts[0], norm_ts_bounds_ts[0]], [Y_0_bounds_ts[1], mass_bounds_ts[1], temp_bounds_ts[1], q_0_bounds_ts[1], R_0_bounds_ts[1], beta_bounds_ts[1], n_0_ts_bounds_ts[1], norm_ts_bounds_ts[1]])

print(f"fitting the data to the boltzmann gibbs blast wave model")
#popt, pcov = curve_fit(boltzmann_gibbs_blast_wave, pT, nine, p0=[m_0, T_0, beta_s_0, R_0, n_0_bg, norm_bg], bounds=all_bounds_bg)

# using least squares method
res_lsq = least_squares(lambda x: boltzmann_gibbs_blast_wave(pT, *x) - nine, [m_0_pi, T_0, beta_s_0, R_0, n_0_bg, norm_bg], bounds=all_bounds_bg, verbose=2)
popt = res_lsq.x

print(f"fitting the data to the tsallis blast wave model")
# popt2, pcov2 = curve_fit(tsallis_blast_wave, pT, nine, p0=[Y_0,  m_0, T_0, q_0, R_0, beta, n_0_ts, norm_ts],bounds=all_bounds_ts)

# using least squares method
iteration_limit = 15
res_lsq = least_squares(lambda x: tsallis_blast_wave(pT, *x) - nine, [Y_0,  m_0_pi, T_0, q_0, R_0, beta, n_0_ts, norm_ts], bounds=all_bounds_ts, verbose=2, max_nfev=iteration_limit)
popt2 = res_lsq.x

fitting the data to the boltzmann gibbs blast wave model
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.9344e+02                                    8.70e+05    
       1              2         7.5081e+01      1.18e+02       2.90e+06       8.60e+02    
       2              3         6.0226e+00      6.91e+01       2.45e+05       9.59e+01    
       3              4         2.1345e+00      3.89e+00       1.40e+05       2.11e+04    
       4              5         2.0044e+00      1.30e-01       4.81e+05       4.35e+04    
       5             10         1.8966e+00      1.08e-01       2.52e+00       5.31e+04    
       6             11         1.5506e+00      3.46e-01       4.61e-01       1.79e+04    
       7             12         1.5450e+00      5.68e-03       4.87e-01       5.60e+04    
       8             13         1.2274e+00      3.18e-01       1.96e-02       7.39e+01    
`xtol` termination condition is s

In [11]:
### ---------- plotting the data and the fits ---------- ###

print(f"Getting the datapoints for the fits")
dN_dpT_bg = boltzmann_gibbs_blast_wave(pT, *popt)
dN_dpT_ts = tsallis_blast_wave(pT, *popt2)

# calculate the chi-squared values for the fits
chi2_bg = np.sum(((boltzmann_gibbs_blast_wave(pT, *popt) - nine)**2) / nine)
chi2_ts = np.sum(((tsallis_blast_wave(pT, *popt2) - nine)**2) / nine)


print(f'Chi-squared value for Boltzmann Gibbs Blast Wave fit: {chi2_bg}')

# plotter
print(f"Plotting the data and the fits")
plt.figure(figsize=(10, 6))
plt.plot(pT, nine, 'o', label='original data')
plt.plot(pT, dN_dpT_bg, '--', label=f'BGBW fit, chi2 = {chi2_bg}')
plt.plot(pT, dN_dpT_ts, '--', label=f'TBW fit, chi2 = {chi2_ts}')
plt.yscale('log')
plt.legend()
plt.xlabel('Transverse momentum (GeV)')
plt.ylabel('dN/dpT')
plt.title('Blast Wave Models')
#plt.savefig('fits_goodresult2_Rchange.png')
plt.show()


Getting the datapoints for the fits


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('float64'), dtype('<U32')) -> None

In [9]:
# Read the CSV file and extract data from the first column, skipping the first row
data = pd.read_csv("D:/ACADEMICS/Coding/Comp Phy/term_paper/kaon_data.csv", usecols=[0,1], skiprows=[0,1,2,3,4,5])
#lim_length = 40
# Convert the extracted data to a list
data_list_k = data.values.tolist()
data_list_k = np.array(data_list)
pT_k = np.array(data_list_k)[:,0].flatten()
nine = np.array(data_list_k)[:,1].flatten()

# changing dtype of pT to float
pT_k = pT_k.astype(float)
nine = nine.astype(float)

print(pT_k.shape)
print(nine.shape)
#print(pT)

ValueError: could not convert string to float: '-'